In [77]:
import pandas as pd
import spacy
import requests
import datetime
import os
from pathlib import Path
from dotenv import load_dotenv
import openai
import anthropic

In [78]:

load_dotenv()

OPENAI_API_KEY      = os.getenv('OPENAI_API_KEY')
PERPLEXITY_API_KEY  = os.getenv('PERPLEXITY_API_KEY')
ANTHROPIC_API_KEY   = os.getenv('ANTHROPIC_API_KEY')
TEST                = os.getenv('TEST')
HUMAN_FILE          = os.getenv('HUMAN_FILE')

print(TEST, HUMAN_FILE)

0 imports/HUMAN-TEXTS-1


In [79]:
oai_client = openai.OpenAI()
ant_client = anthropic.Anthropic()



In [80]:
def export_full_text(tl, filename, marker='\n--------------------NEW-TEXT------------------\n'):
    with open(filename, 'w') as f:
        f.write(marker.join(tl))

In [81]:
def gpt4o_request(prompt, system_prompt):
    completion = oai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "text"
        },
        temperature=1,
        max_tokens=1993,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    return str(completion.choices[0].message.content)

def gpt35t_request(prompt, system_prompt):
    completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "text"
        },
        temperature=1,
        max_tokens=1993,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    return str(completion.choices[0].message.content)

def perplxty_request(prompt, system_prompt):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json",
    }
    data = {
        "model": "llama-3.1-sonar-large-128k-online",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1993,
        "temperature": 0.5,
        "top_p": 1
    }

    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"].strip()

def clde_request(prompt, system_prompt):
    message = ant_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1993,
        temperature=0.5,
        system=system_prompt,  # Top-level system parameter
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
    )
    return message.content[0].text


In [82]:
prompt          = []
system_prompt   = ''
gpt4o_texts     = []
gpt35t_texts    = []
perplxty_texts  = []
clde_texts      = []

In [83]:
if int(TEST) == 1:   
    with open('imports/TEST-1', 'r') as f:
        human_texts = f.read()
else:
    with open(HUMAN_FILE, 'r') as f:
        human_texts = f.read()

human_texts = human_texts.split('--------------------NEW-TEXT------------------')

for i in human_texts:
    human_txt_list = i.split()
    prompt.append(f'Schreibe einen Fußballartikel zum Thema WM Sieg der deutschen National Mannschaft am 13.07.2014! Achten Sie auf fachliche korrektness und auf die im Systemprompt angegebenen Schemata! Der Text beginnt mit: {human_txt_list[0], human_txt_list[1], human_txt_list[2]}. Folgen Sie diesem Beginn! Der Artikel soll mindestens 700 und maximal 800 Wörter lang sein!')
    

In [84]:
l = []
for i in human_texts:
    human_txt_list = i.split()
    l.append(f'{human_txt_list[0]}, {human_txt_list[1]}, {human_txt_list[2]}')
    

In [85]:
print(l)

['So, reden, große', 'Die, deutsche, Fußball-Nationalelf', 'Es, ist, vollbracht:', 'Deutschland, ist, Weltmeister!', 'Drei, Minuten, vor', 'Da, ist, der', 'Die, DFB-Elf, gewinnt', 'Weltmeister!, Deutschland, ist', 'Die, DFB-Elf, hat', 'Der, eingewechselte, Mario', 'Geschafft!, Die, Deutsche', 'Rio, de, Janeiro', 'Deutschland, hatte, zunächst', 'Am, 13., Juli', 'Mit, einem, mühevollen', 'Nach, einem, dramatischen', 'Weltmeister!, Zum, ersten', '"Schlaaaaand!!!", Hunderttausende, Fußballfans', 'Obschon, der, lange', 'Dieses, Spiel!, Diese']


In [66]:
counter = 0
Path('backups').mkdir(parents=True, exist_ok=True)
for i in prompt:
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    
    perplxty_texts.append(perplxty_request(i, system_prompt))
    open(f'backups/perplexity_{timestamp}_{counter}.txt', 'w').write(perplxty_texts[- 1])
    print(f'Perplexity: {counter}')

    counter += 1

Perplexity: 0
Perplexity: 1
Perplexity: 2
Perplexity: 3
Perplexity: 4
Perplexity: 5
Perplexity: 6
Perplexity: 7
Perplexity: 8
Perplexity: 9
Perplexity: 10
Perplexity: 11
Perplexity: 12
Perplexity: 13
Perplexity: 14
Perplexity: 15
Perplexity: 16
Perplexity: 17
Perplexity: 18
Perplexity: 19


In [87]:
counter = 0
Path('imports').mkdir(parents=True, exist_ok=True)
Path('backups').mkdir(parents=True, exist_ok=True)

for i in prompt:
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    gpt4o_texts.append(gpt4o_request(i, system_prompt))
    print(f'GPT-4o: {counter}')
    gpt35t_texts.append(gpt35t_request(i, system_prompt))
    print(f'GPT-3.5-turbo: {counter}')
    perplxty_texts.append(perplxty_request(i, system_prompt))
    print(f'Perplexity: {counter}')
    clde_texts.append(clde_request(i, system_prompt))
    print(f'CLDE: {counter}')
    
    counter += 1

GPT-4o: 0
GPT-3.5-turbo: 0
Perplexity: 0
CLDE: 0
GPT-4o: 1
GPT-3.5-turbo: 1
Perplexity: 1
CLDE: 1
GPT-4o: 2
GPT-3.5-turbo: 2
Perplexity: 2
CLDE: 2
GPT-4o: 3
GPT-3.5-turbo: 3
Perplexity: 3
CLDE: 3
GPT-4o: 4
GPT-3.5-turbo: 4
Perplexity: 4
CLDE: 4
GPT-4o: 5
GPT-3.5-turbo: 5
Perplexity: 5
CLDE: 5
GPT-4o: 6
GPT-3.5-turbo: 6
Perplexity: 6
CLDE: 6
GPT-4o: 7
GPT-3.5-turbo: 7
Perplexity: 7
CLDE: 7
GPT-4o: 8
GPT-3.5-turbo: 8
Perplexity: 8
CLDE: 8
GPT-4o: 9
GPT-3.5-turbo: 9
Perplexity: 9
CLDE: 9
GPT-4o: 10
GPT-3.5-turbo: 10
Perplexity: 10
CLDE: 10
GPT-4o: 11
GPT-3.5-turbo: 11
Perplexity: 11
CLDE: 11
GPT-4o: 12
GPT-3.5-turbo: 12
Perplexity: 12
CLDE: 12
GPT-4o: 13
GPT-3.5-turbo: 13
Perplexity: 13
CLDE: 13
GPT-4o: 14
GPT-3.5-turbo: 14
Perplexity: 14
CLDE: 14
GPT-4o: 15
GPT-3.5-turbo: 15
Perplexity: 15
CLDE: 15
GPT-4o: 16
GPT-3.5-turbo: 16
Perplexity: 16
CLDE: 16
GPT-4o: 17
GPT-3.5-turbo: 17
Perplexity: 17
CLDE: 17
GPT-4o: 18
GPT-3.5-turbo: 18
Perplexity: 18
CLDE: 18
GPT-4o: 19
GPT-3.5-turbo: 19
Perp

In [88]:
for idx, i in enumerate(perplxty_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    perplxty_texts[idx] = new_text

In [89]:
for idx, i in enumerate(clde_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    clde_texts[idx] = new_text

In [90]:
for idx, i in enumerate(gpt4o_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    gpt4o_texts[idx] = new_text

In [91]:
for idx, i in enumerate(gpt35t_texts):
    char_list = [*i]
    result = []
    skip = 0

    for j in char_list:
        if skip > 0:
            skip -= 1
            continue

        if j == '[':
            skip = 2  # Überspringe die nächsten 2 Zeichen
        elif j == '*':
            skip = 0  # Überspringe das nächste Zeichen
        elif j == '#':
            skip = 0
        else:
            result.append(j)

    new_text = ''.join(result)
    gpt35t_texts[idx] = new_text

In [92]:
marker = '\n--------------------NEW-TEXT------------------\n'

perplxty_texts_str = marker.join(perplxty_texts)
gpt4o_texts_str = marker.join(gpt4o_texts)
gpt35t_texts_str = marker.join(gpt35t_texts)
clde_texts_str = marker.join(clde_texts)
human_texts_str = marker.join(human_texts)

with open('exports/perplexity_full.txt', 'w') as f:
    f.write(perplxty_texts_str)

with open('exports/gpt4o_full.txt', 'w') as f:
    f.write(gpt4o_texts_str)

with open('exports/gpt35t_full.txt', 'w') as f:
    f.write(gpt35t_texts_str)

with open('exports/clde_full.txt', 'w') as f:
    f.write(clde_texts_str)

with open('exports/human_full.txt', 'w') as f:
    f.write(human_texts_str)

In [94]:
gpt4o_df    = pd.DataFrame({"words": []})
gpt35t_df   = pd.DataFrame({"words": []})
gemni_df    = pd.DataFrame({"words": []})
perplxty_df = pd.DataFrame({"words": []})
clde_df     = pd.DataFrame({"words": []})
human_df    = pd.DataFrame({"words": []})

In [95]:
nlp = spacy.load('de_core_news_sm')

In [96]:
for txt in gpt4o_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    gpt4o_df = pd.concat([gpt4o_df,txt_df], ignore_index=True)


In [97]:

for txt in gpt35t_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    gpt35t_df = pd.concat([gpt35t_df,txt_df], ignore_index=True)


In [98]:

for txt in perplxty_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    perplxty_df = pd.concat([perplxty_df,txt_df], ignore_index=True)


In [99]:

for txt in clde_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    clde_df = pd.concat([clde_df,txt_df], ignore_index=True)


In [100]:

for txt in human_texts:
    txt_lem = nlp(txt)
    txt_df  = pd.DataFrame([token.lemma_ for token in txt_lem], columns=['words'])
    human_df = pd.concat([human_df,txt_df], ignore_index=True)

In [101]:
gpt4o_df    = gpt4o_df.value_counts().reset_index()
gpt35t_df   = gpt35t_df.value_counts().reset_index()
perplxty_df = perplxty_df.value_counts().reset_index()
clde_df     = clde_df.value_counts().reset_index()
human_df    = human_df.value_counts().reset_index()

gpt4o_df.columns    = ['words', 'amount']
gpt35t_df.columns   = ['words', 'amount']
perplxty_df.columns = ['words', 'amount']
clde_df.columns     = ['words', 'amount']
human_df.columns    = ['words', 'amount']

In [102]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
Path('exports').mkdir(parents=True, exist_ok=True)

gpt4o_df.to_csv(f'exports/GPT4o-{timestamp}.csv', index=True)
gpt35t_df.to_csv(f'exports/GPT35t-{timestamp}.csv', index=True)
perplxty_df.to_csv(f'exports/PERPLEXITY-{timestamp}.csv', index=True)
clde_df.to_csv(f'exports/CLAUDE-{timestamp}.csv', index=True)
human_df.to_csv(f'exports/HUMAN-{timestamp}.csv', index=True)